# NFL SCRAPER

This notebook generates data from all NFL games from 2012 to current year. It then calculates the associated ELOs for each team for the start of the current season based on the game performance. The NFL Predictor notebook will generate predictions for the next seasons games.

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import math
from scipy import stats
import random

In [6]:
#This cell generates all basic game data (scores, yards, etc)
#Only run if you want to scrape the data, otherwise import excel file which has the scraped data already saved.

#Parts of this cell created and adapted from:
#https://github.com/Degentleman/NFL-Results-Scraper/blob/master/NFLHistoricalTeamData.py

# URL to Format
url_template = "https://www.pro-football-reference.com/years/{year}/games.htm"

# Iterate Player Data Frame for Each Year Specified
nfl_df = pd.DataFrame()

for year in range(2012, 2021):
    url = url_template.format(year=year)  # get the url
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    
    column_headers = [th.getText() for th in soup.findAll('thead', limit=1)[0].findAll('th')]    
    data_rows = soup.findAll('tbody', limit=1)[0].findAll('tr')[0:]
    player_data = [[td.getText() for td in data_rows[i].findAll(['th','td'])] for i in range(len(data_rows))]
    year_df = pd.DataFrame(player_data, columns=column_headers)
    year_df['Year'] = year
    year_df = year_df[(year_df.Date !='Playoffs') & (year_df.Week != 'Week')]
    
    boxscorelinks = []
    for i in range(len(data_rows)):
        for link in data_rows[i].find_all('a', text = 'boxscore'):
            boxscorelinks = boxscorelinks + [link.get('href')]
    boxscoreDF = pd.DataFrame(boxscorelinks)

    year_df.reset_index(drop=True, inplace=True)
    
    year_df = pd.concat( [year_df, boxscoreDF], axis=1) 
    
    nfl_df = pd.concat([nfl_df,year_df],axis=0)

nfl_df = nfl_df.reset_index()
nfl_df = nfl_df.drop(['index'], axis = 1)
nfl_df.columns = [*nfl_df.columns[:-1], 'BoxscoreURL']
nfl_df.dropna(inplace = True)
nfl_df.to_excel("Nfl_Scraper.xlsx", sheet_name = 'NFL_DF') #Saves the scraped data to excel

In [31]:
#This cell generates game specific data
#Only run if you want to scrape data, otherwise import excel file which has been saved.

game_url_template = "https://www.pro-football-reference.com/{boxscoreURL}"
player_game_rating = pd.DataFrame()
for game in range(0,len(nfl_df)):
    boxscoreURL = nfl_df.iloc[game]['BoxscoreURL']
    game_url = game_url_template.format(boxscoreURL=boxscoreURL)
    html = urlopen(game_url)
    soup = BeautifulSoup(html, 'html.parser')
    
    qb1_name = [th.getText() for th in soup.findAll('tbody', limit=3)[2].findAll('a')][0]
    qb1_team = [th.getText() for th in soup.findAll('tbody', limit=3)[2].findAll('td')][0]
    qb1_rating = soup.findAll('tbody', limit=3)[2].findAll('td')[9].getText()
    
    #Finds the first row of the opposing team players
    #This will not count for substitutions as game predictions are assumed to have one quarterback to play
    for a in range(0,20):
        qb2_rowindex = 21*a
        if(soup.findAll('tbody', limit=3)[2].findAll('td')[qb2_rowindex].getText() != qb1_team):
            break
    
    qb2_name = [th.getText() for th in soup.findAll('tbody', limit=3)[2].findAll('a')][int(qb2_rowindex/21)]
    qb2_team = [th.getText() for th in soup.findAll('tbody', limit=3)[2].findAll('td')][qb2_rowindex]
    qb2_rating = soup.findAll('tbody', limit=3)[2].findAll('td')[qb2_rowindex + 9].getText()
    
    qb_name = [qb1_name, qb2_name]
    qb_team = [qb1_team, qb2_team]
    qb_rating = [qb1_rating, qb2_rating]
    
  
    game_rating = pd.DataFrame(zip([boxscoreURL, boxscoreURL],qb_name, qb_team, qb_rating), columns = ['GameURL', 'Quarterback', 'Team', 'Rating'])
    
    player_game_rating = pd.concat([player_game_rating, game_rating], axis = 0)
    if(game%100 == 0):
        print(game)
with pd.ExcelWriter('NFL_Scraper.xlsx',
                    mode='a', engine ="openpyxl") as writer:  
    player_game_rating.to_excel(writer, sheet_name='QB_Game_Rating')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [12]:
#START HERE IF YOU HAVE EXCEL FILES DOWNLOADED

nfl_df = pd.read_excel('NFL_Scraper.xlsx', sheet_name = 'NFL_DF')
player_game_rating = pd.read_excel('NFL_Scraper.xlsx', sheet_name = 'QB_Game_Rating')

nfl_df.drop(nfl_df.columns[0], axis = 1, inplace = True)
player_game_rating.drop(player_game_rating.columns[0], axis = 1, inplace = True)
player_game_rating['Quarterback'] = player_game_rating['Quarterback'].str.rstrip()

In [13]:
player_game_rating

,GameURL,Quarterback,Team,Rating
0,/boxscores/201209050nyg.htm,Tony Romo,DAL,129.5
1,/boxscores/201209050nyg.htm,Eli Manning,NYG,94.9
2,/boxscores/201209090chi.htm,Andrew Luck,IND,52.9
3,/boxscores/201209090chi.htm,Jay Cutler,CHI,98.9
4,/boxscores/201209090htx.htm,Ryan Tannehill,MIA,39.0
...,...,...,...,...
5109,/boxscores/202111070kan.htm,Patrick Mahomes,KAN,74.8
5110,/boxscores/202111070ram.htm,Ryan Tannehill,TEN,79.7
5111,/boxscores/202111070ram.htm,Matthew Stafford,LAR,71.0
5112,/boxscores/202111080pit.htm,Justin Fields,CHI,89.9


In [14]:
#Initialize QB dictionary with their QBR
qb_dict = {}

In [15]:
#This cell will create updated quarterback ratings in a dictionary
def qb_dict_update(game):
    if(game[1] not in qb_dict.keys()):
        qb_dict[game[1]] = game[3]
    else:
        qb_dict[game[1]] = (1/6)*float(game[3]) + (5/6)*float(qb_dict.get(game[1]))

In [16]:
player_game_rating.head()

,GameURL,Quarterback,Team,Rating
0,/boxscores/201209050nyg.htm,Tony Romo,DAL,129.5
1,/boxscores/201209050nyg.htm,Eli Manning,NYG,94.9
2,/boxscores/201209090chi.htm,Andrew Luck,IND,52.9
3,/boxscores/201209090chi.htm,Jay Cutler,CHI,98.9
4,/boxscores/201209090htx.htm,Ryan Tannehill,MIA,39.0


In [17]:
#Add all NFL teams to a dictionary
teams = ['Arizona Cardinals',
'Atlanta Falcons',
'Baltimore Ravens',
'Buffalo Bills',
'Carolina Panthers',
'Chicago Bears',
'Cincinnati Bengals',
'Cleveland Browns',
'Dallas Cowboys',
'Denver Broncos',
'Detroit Lions',
'Green Bay Packers',
'Houston Texans',
'Indianapolis Colts',
'Jacksonville Jaguars',
'Kansas City Chiefs',
'Las Vegas Raiders',
'Los Angeles Chargers',
'Los Angeles Rams',
'Miami Dolphins',
'Minnesota Vikings',
'New England Patriots',
'New Orleans Saints',
'New York Giants',
'New York Jets',
'Philadelphia Eagles',
'Pittsburgh Steelers',
'San Francisco 49ers',
'Seattle Seahawks',
'Tampa Bay Buccaneers',
'Tennessee Titans',
'Washington Football Team']
team_elo = pd.DataFrame(teams, columns = ['Team'])
team_elo = team_elo.set_index('Team')
team_dict = dict([(y,x+1) for x,y in enumerate(sorted(set(teams)))])
#Add location changed teams
team_dict['San Diego Chargers'] = 18
team_dict['Oakland Raiders'] = 17
team_dict['St. Louis Rams'] = 19
team_dict['Washington Redskins'] = 32
#Initial ELO 
team_elo['elo'] = 1200

In [18]:
#These two functions are purely made to go through the past data and to make adjustments to the elo to determine an accurate current ELO number for each team.
def winning_elo_adjustment(game):
    elo_adjustment = 0
    if(nfl_df.iloc[game][5] == '' or pd.isnull(nfl_df.iloc[game,5])):
        #Home team advantage
        qb_dict_update(player_game_rating.iloc[2*game+1])
        elo_adjustment = elo_adjustment + 30 + qb_dict.get(player_game_rating.iloc[2*game+1][1])
    
    else:
        qb_dict_update(player_game_rating.iloc[2*game])
        elo_adjustment = elo_adjustment + qb_dict.get(player_game_rating.iloc[2*game][1])
    
    return elo_adjustment

def losing_elo_adjustment(game):
    elo_adjustment = 0
    if(nfl_df.iloc[game][5] == '@'):
        #Home team advantage
        qb_dict_update(player_game_rating.iloc[2*game+1])
        elo_adjustment = elo_adjustment + 30 + qb_dict.get(player_game_rating.iloc[2*game+1][1])

    else:
        qb_dict_update(player_game_rating.iloc[2*game])
        elo_adjustment = elo_adjustment + qb_dict.get(player_game_rating.iloc[2*game][1])
    
    return elo_adjustment

In [19]:
CurrentYear = 2021
for game in range(0,len(nfl_df)):
    #Elo System Design inspired by: https://fivethirtyeight.com/methodology/how-our-nfl-predictions-work/
    #The name of the team references the team dictionary, which provides the reference value for the team's elo value.
    #This was included to account for name changes of teams. For the sake of the ELO measurement, a team that switches location will retain the same ELO
    winning_team_elo = team_elo.iloc[team_dict[nfl_df.iloc[game][4]]-1]['elo']
    losing_team_elo = team_elo.iloc[team_dict[nfl_df.iloc[game][6]]-1]['elo']
    
    #ELO Adjustments based on different conditions
    winning_team_adjustment = winning_elo_adjustment(game)
    losing_team_adjustment = losing_elo_adjustment(game)
    
    elo_diff = (winning_team_elo + winning_team_adjustment) - (losing_team_elo + losing_team_adjustment)
    
    #Probability of the winning team winning, before ELO adjustment
    prob_win = 1/((10**(-1*elo_diff/400)) + 1)
    forecast_delta = 1-prob_win
    k_factor = 20
    
    winner_point_diff = int(nfl_df.iloc[game][8]) - int(nfl_df.iloc[game][9])
    mov_multiplier = math.log(winner_point_diff + 1) * (2.2)/((elo_diff * 0.001)+2.2)  
    if(nfl_df.iloc[game]['Year'] == CurrentYear):
        mov_multiplier *= 1.5
        
    elo_change = mov_multiplier * forecast_delta * k_factor
    
    team_elo.iloc[team_dict[nfl_df.iloc[game][4]]-1]['elo'] = team_elo.iloc[team_dict[nfl_df.iloc[game][4]]-1]['elo'] + elo_change
    team_elo.iloc[team_dict[nfl_df.iloc[game][6]]-1]['elo'] = team_elo.iloc[team_dict[nfl_df.iloc[game][6]]-1]['elo'] - elo_change
    

In [20]:
player_game_rating.head()

,GameURL,Quarterback,Team,Rating
0,/boxscores/201209050nyg.htm,Tony Romo,DAL,129.5
1,/boxscores/201209050nyg.htm,Eli Manning,NYG,94.9
2,/boxscores/201209090chi.htm,Andrew Luck,IND,52.9
3,/boxscores/201209090chi.htm,Jay Cutler,CHI,98.9
4,/boxscores/201209090htx.htm,Ryan Tannehill,MIA,39.0


In [21]:
#Ordered Teams by ELO
team_elo.sort_values('elo', ascending = False )

,elo
Team,
Tampa Bay Buccaneers,1331
Green Bay Packers,1313
Arizona Cardinals,1300
Tennessee Titans,1300
Buffalo Bills,1293
New Orleans Saints,1277
Kansas City Chiefs,1272
Los Angeles Rams,1259
Baltimore Ravens,1256


Now that we have all the team's ELO, we need to create a dictionary of the current starting quarterbacks and their associated teams. We also need to add a script that will update the ELO's with current season data as they happen.

In [22]:
#As quarterback movement changes freqeuntly, the current quarterback for each team is always changing. The list of current quarterbacks is imported from an updated excel sheet.

current_qb = pd.read_excel('QB_List.xlsx', sheet_name = 'Current_QB')
current_qb['Current QBR'] = current_qb.apply(lambda row: qb_dict.get(row['Quarterback']), axis = 1)
current_qb = current_qb.set_index('Team')
current_qb

,Quarterback,Current QBR
Team,,
Arizona Cardinals,Kyler Murray,104.270360
Atlanta Falcons,Matt Ryan,93.839512
Baltimore Ravens,Lamar Jackson,93.053726
Buffalo Bills,Josh Allen,106.008897
Carolina Panthers,Sam Darnold,71.859057
Chicago Bears,Justin Fields,60.036755
Cincinnati Bengals,Joe Burrow,105.663405
Cleveland Browns,Baker Mayfield,92.443257
Dallas Cowboys,Dak Prescott,112.149591


In [24]:
all_qb_df = pd.DataFrame(qb_dict.items(), columns = ['Quarterback', 'Rating'])

In [25]:
with pd.ExcelWriter('NFL_Scraper.xlsx',
                    mode='a', engine ="openpyxl") as writer:  
    current_qb.to_excel(writer, sheet_name='Current_Starting_QBR')
    
with pd.ExcelWriter('NFL_Scraper.xlsx',
                    mode='a', engine ="openpyxl") as writer:  
    pd.DataFrame(qb_dict.items(), columns = ['Quarterback', 'Rating']).to_excel(writer, sheet_name='All_QBR')
    
with pd.ExcelWriter('NFL_Scraper.xlsx',
                    mode='a', engine ="openpyxl") as writer:  
     team_elo.to_excel(writer, sheet_name='Team_ELO')

In [2]:
#This cell generates all basic game data (scores, yards, etc)
#Only run if you want to scrape the data, otherwise import excel file which has the scraped data already saved.

#Parts of this cell created and adapted from:
#https://github.com/Degentleman/NFL-Results-Scraper/blob/master/NFLHistoricalTeamData.py

# URL to Format
url_template = "https://www.pro-football-reference.com/years/{year}/games.htm"

# Iterate Player Data Frame for Each Year Specified
nfl_df = pd.DataFrame()

for year in range(2020, 2022):
    url = url_template.format(year=year)  # get the url
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    
    column_headers = [th.getText() for th in soup.findAll('thead', limit=1)[0].findAll('th')]    
    data_rows = soup.findAll('tbody', limit=1)[0].findAll('tr')[0:]
    player_data = [[td.getText() for td in data_rows[i].findAll(['th','td'])] for i in range(len(data_rows))]
    year_df = pd.DataFrame(player_data, columns=column_headers)
    year_df['Year'] = year
    year_df = year_df[(year_df.Date !='Playoffs') & (year_df.Week != 'Week')]
    
    boxscorelinks = []
    for i in range(len(data_rows)):
        for link in data_rows[i].find_all('a', text = 'boxscore'):
            boxscorelinks = boxscorelinks + [link.get('href')]
    boxscoreDF = pd.DataFrame(boxscorelinks)

    year_df.reset_index(drop=True, inplace=True)
    
    year_df = pd.concat( [year_df, boxscoreDF], axis=1) 
    
    nfl_df = pd.concat([nfl_df,year_df],axis=0)

nfl_df = nfl_df.reset_index()
nfl_df = nfl_df.drop(['index'], axis = 1)
nfl_df.columns = [*nfl_df.columns[:-1], 'BoxscoreURL']
nfl_df.dropna(inplace = True)
#nfl_df.to_excel("Nfl_ScraperNew.xlsx", sheet_name = 'NFL_DF') #Saves the scraped data to excel

In [3]:
add_on = nfl_df[(nfl_df['Year'] == 2021) & (nfl_df['Week'] == '9')]

In [4]:
#add_on = nfl_df[(nfl_df['Year'] == 2020) & (nfl_df['Week'] == 'SuperBowl')]

In [5]:
nfl_df = pd.read_excel('NFL_Scraper.xlsx', sheet_name = 'NFL_DF')
player_game_rating = pd.read_excel('NFL_Scraper.xlsx', sheet_name = 'QB_Game_Rating')

nfl_df.drop(nfl_df.columns[0], axis = 1, inplace = True)
player_game_rating.drop(player_game_rating.columns[0], axis = 1, inplace = True)
player_game_rating['Quarterback'] = player_game_rating['Quarterback'].str.rstrip()

In [6]:
nfl_df = nfl_df.rename({'Unnamed: 6' : '', 'Unnamed: 8': ''}, axis = 1)

In [7]:
nfl_df = nfl_df.append(add_on, ignore_index = True)

In [8]:
nfl_df[(nfl_df['Year'] == 2021) & (nfl_df['Week'] == '9')]

,Week,Day,Date,Time,Winner/tie,,Loser/tie,,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Year,BoxscoreURL
2527,9,Thu,2021-11-04,8:20PM,Indianapolis Colts,,New York Jets,boxscore,45,30,532,0,486,2,2021,/boxscores/202111040clt.htm
2528,9,Sun,2021-11-07,1:00PM,Baltimore Ravens,,Minnesota Vikings,boxscore,34,31,500,2,318,0,2021,/boxscores/202111070rav.htm
2529,9,Sun,2021-11-07,1:00PM,Atlanta Falcons,@,New Orleans Saints,boxscore,27,25,366,0,376,1,2021,/boxscores/202111070nor.htm
2530,9,Sun,2021-11-07,1:00PM,Jacksonville Jaguars,,Buffalo Bills,boxscore,9,6,218,1,301,3,2021,/boxscores/202111070jax.htm
2531,9,Sun,2021-11-07,1:00PM,Denver Broncos,@,Dallas Cowboys,boxscore,30,16,407,0,290,2,2021,/boxscores/202111070dal.htm
2532,9,Sun,2021-11-07,1:00PM,New York Giants,,Las Vegas Raiders,boxscore,23,16,247,1,403,3,2021,/boxscores/202111070nyg.htm
2533,9,Sun,2021-11-07,1:00PM,Cleveland Browns,@,Cincinnati Bengals,boxscore,41,16,361,0,348,3,2021,/boxscores/202111070cin.htm
2534,9,Sun,2021-11-07,1:00PM,New England Patriots,@,Carolina Panthers,boxscore,24,6,273,2,240,3,2021,/boxscores/202111070car.htm
2535,9,Sun,2021-11-07,1:00PM,Miami Dolphins,,Houston Texans,boxscore,17,9,262,5,272,4,2021,/boxscores/202111070mia.htm
2536,9,Sun,2021-11-07,4:05PM,Los Angeles Chargers,@,Philadelphia Eagles,boxscore,27,24,445,0,331,0,2021,/boxscores/202111070phi.htm


In [9]:
#This cell generates game specific data
#Only run if you want to scrape data, otherwise import excel file which has been saved.

game_url_template = "https://www.pro-football-reference.com/{boxscoreURL}"
player_game_rating_addon = pd.DataFrame()
for game in range(2527,2541):
    boxscoreURL = nfl_df.iloc[game]['BoxscoreURL']
    game_url = game_url_template.format(boxscoreURL=boxscoreURL)
    html = urlopen(game_url)
    soup = BeautifulSoup(html, 'html.parser')
    
    qb1_name = [th.getText() for th in soup.findAll('tbody', limit=3)[2].findAll('a')][0]
    qb1_team = [th.getText() for th in soup.findAll('tbody', limit=3)[2].findAll('td')][0]
    qb1_rating = soup.findAll('tbody', limit=3)[2].findAll('td')[9].getText()
    
    #Finds the first row of the opposing team players
    #This will not count for substitutions as game predictions are assumed to have one quarterback to play
    for a in range(0,20):
        qb2_rowindex = 21*a
        if(soup.findAll('tbody', limit=3)[2].findAll('td')[qb2_rowindex].getText() != qb1_team):
            break
    
    qb2_name = [th.getText() for th in soup.findAll('tbody', limit=3)[2].findAll('a')][int(qb2_rowindex/21)]
    qb2_team = [th.getText() for th in soup.findAll('tbody', limit=3)[2].findAll('td')][qb2_rowindex]
    qb2_rating = soup.findAll('tbody', limit=3)[2].findAll('td')[qb2_rowindex + 9].getText()
    
    qb_name = [qb1_name, qb2_name]
    qb_team = [qb1_team, qb2_team]
    qb_rating = [qb1_rating, qb2_rating]
    
  
    game_rating = pd.DataFrame(zip([boxscoreURL, boxscoreURL],qb_name, qb_team, qb_rating), columns = ['GameURL', 'Quarterback', 'Team', 'Rating'])
    
    player_game_rating_addon = pd.concat([player_game_rating_addon, game_rating], axis = 0)

In [10]:
player_game_rating = player_game_rating.append(player_game_rating_addon, ignore_index = True)

In [11]:
nfl_df.to_excel("Nfl_Scraper.xlsx", sheet_name = 'NFL_DF') #Saves the scraped data to excel

with pd.ExcelWriter('NFL_Scraper.xlsx',
                    mode='a', engine ="openpyxl") as writer:  
    player_game_rating.to_excel(writer, sheet_name='QB_Game_Rating')